In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import os
import random
import glob
import pickle
import tqdm
from models import *
from keras.callbacks import EarlyStopping

2024-03-28 19:17:56.791783: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 19:17:57.421688: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def generator(IDs, yields, batch_size, cutoff=None):
    import numpy as np
    import random
    
 # Create empty arrays to contain batch of features and labels#

    if cutoff != None:
        batch_features = np.zeros((batch_size, cutoff, 1, 128, 9))
        batch_yields = np.zeros((batch_size))
        while True:
            for i in range(batch_size):
                # choose random index in features
                index = random.choice(range(len(IDs)))
                ID = IDs[index]
                if np.sum(np.isnan(np.load('data/PROCESSED_III/' + ID + '.npy'))) == 0:
                    batch_features[i, :, :, :, :] = np.load('data/PROCESSED_III/' + ID + '.npy')[:cutoff, :, :, :]
                    #print('yes', ID)
                    batch_yields[i] = yields[ID]
                else:
                    print('no', ID)
                    
            yield batch_features, batch_yields
                    
    else:
        batch_features = np.zeros((batch_size, 38, 1, 128, 9))
        batch_yields = np.zeros((batch_size))
        while True:
            for i in range(batch_size):
                # choose random index in features
                index = random.choice(range(len(IDs)))
                ID = IDs[index]
                if np.sum(np.isnan(np.load('data/PROCESSED_III/' + ID + '.npy'))) == 0:
                    batch_features[i, :, :, :, :] = np.load('data/PROCESSED_III/' + ID + '.npy')
                    #print('yes', ID)
                    batch_yields[i] = yields[ID]
                else:
                    print('no', ID)
            yield batch_features, batch_yields

In [3]:
# training model on data of year 2010-2015 (6 years total)

model_list = ['CNN_LSTM', 'SepCNN_LSTM', 'CONVLSTM', 'CONV3D', 'CONVLSTM_CONV3D']

# Datasets
yields = pickle.load(open('data/yields.p', 'rb'))
y = yields
print(len(yields['train']), len(yields['validation']))

# define early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, \
                          verbose=1, mode='auto')
callbacks_list = [earlystop]

# Generators
training_generator = generator(list(y['train'].keys()), y['train'], 16)
validation_generator = generator(list(y['validation'].keys()), y['validation'], 16)

for model_name in model_list:
    rm = ResearchModels(model_name, 38, (1, 128, 9), print_model=True)
    rm.model.fit_generator(training_generator, validation_data=validation_generator, callbacks=callbacks_list,\
                               validation_steps=1353/16, samples_per_epoch=50, nb_epoch=100, verbose=0)
    rm.save(model_name)

6559 1511
input shape (38, 1, 128, 9)
Loading Model.
Frames Input Shape: (16, 38, 1, 128, 9)


/home/kaywee/Documents/wee's things/cds-corn-prediction/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
2024-03-28 19:17:57.937411: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-28 19:17:57.975604: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the re

NotImplementedError: Exception encountered when calling TimeDistributed.call().

[1mLayer Sequential should implement `def compute_output_shape(self, input_shape)`.[0m

Arguments received by TimeDistributed.call():
  • args=('<KerasTensor shape=(16, 38, 1, 128, 9), dtype=float32, sparse=None, name=keras_tensor>',)
  • kwargs={'mask': 'None'}